# Data Pipeline for C100 Fault Classification 
_February 24, 2020_

In [1609]:
# used to get an idea of run time
from datetime import datetime
startTime = datetime.now()

The purpose of this notebook is to put together a rough impementation of a data pipeline for CEBAF C100 cavity and fault detection. There are two questions that need to be addressed:
<br>

  - *which cavity tripped first?* <br>
  - *which type of fault caused the trip?*

This implementation uses machine learning (as opposed to deep learning) and suffers from the fact that feature extraction and selection is computationally expensive. Nevertheless, it provides the opportunity to implement a functional pipeline on a timescale that can be used for the winter 2020 running period.

## Which Cavity?

At the onset of a cavity trip, the waveform harvester will collect and write data to file. The data collected will be 17 RF signals for each of the 8 cavities in the offending C100 cryomodule. To make the feature engineering manageable (i.e. less computationally expensive) we do the following:

- keep only 4 of the 17 signals for each of the 8 cavities
    - based on analysis from subject matter experts, the most relevant signals are (GASK, GMES, CRFP, DETA2)
- for each signal use `statsmodels` to compute autoregressive features
    - still have not figured out why features computed by `tsfresh` are not effective

### Reading Data

At present I am mimicking the file structure for the data as it appears in `M:\asd\asddata\FCCWaveforms\Spring 2018\rf`. Therefore the script to read in the data will need to be modified.

In [1610]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from pathlib import Path

Create a `dictionary` to use consistent nomenclature for the cryomodule name.

In [1611]:
cav_dict = {'0L04': 'R04', '1L22': 'R1M', '1L23': 'R1N', '1L24': 'R1O', '1L25': 'R1P',
            '1L26': 'R1Q', '2L22': 'R2M', '2L23': 'R2N', '2L24': 'R2O', '2L25': 'R2P', '2L26': 'R2Q'}

cavity_df = pd.DataFrame()
module_path = Path('D:/RF WAVEFORMS/rfw_tsf_extractor-Spring-2019/waveform-data/rf')
dir = Path('D:/RF WAVEFORMS/rfw_tsf_extractor-Spring-2019/labeled-examples')

For developing machine learning models, a list of `fault_*.txt` files were read in. Each file was generated by a subject matter expert and contained a list of trip events. For each event, the cavity and fault label were given as well as the timestamp associated with the fault. As a test of the system, we read in a similar file containing only one event and ignore the labels.

In [1612]:
filelist = ['example.txt']

The code below is ugly but the idea is that after reading in and parsing the timestamp from the file above, it searches the directories and collects the appropriate cavity signals and stores the labels (the cavity number which tripped). With the real-time system we will not have labels (all actions associated with `y` have been commented out).

In [1613]:
#y = pd.Series([])

k = 0
for i in filelist:
    data_file_path = dir/i # IMPORTANT to add sub-directores to `module_path`

    log = pd.read_table(data_file_path, sep='\t')

    m, n = log.shape
    
    for j in range(0, m):
        k += 1
        date, time = log.time[j].split(" ", 1)
        
        # formatting the timestamp in the .txt file to match the format used in the filenames
        date_format = date.replace("/", "_")
        time_format = time.replace(":", "")

        list1 = [time_format, '.', '?']

        ct = os.path.join(module_path, log.zone[j], date_format, "".join(list1))
        
        # output of glob.glob() is a list; rather than convert to string and remove brackets and quotes, select first element
        dir1 = glob.glob(ct)
        dir2 = os.listdir(dir1[0]) # list of filenames in the directory
    
        module_df = pd.DataFrame()
        
        # only read in data if all 8 cavity files are present in directory
        if len(dir2) == 8:

            for m in range(0,8):
                f = os.path.join(dir1[0], dir2[m])
                df = pd.read_table(f, sep='\t')
                sLength = len(df['Time'])
                tStep = (df.Time[2] - df.Time[1]) # in milliseconds
                df['id'] = pd.Series(k, index=df.index)
                col = ['Time', 
                      f'{m+1}_IMES', f'{m+1}_QMES', f'{m+1}_GMES', f'{m+1}_PMES', f'{m+1}_IASK', f'{m+1}_QASK', 
                      f'{m+1}_GASK', f'{m+1}_PASK', f'{m+1}_CRFP', f'{m+1}_CRFPP', f'{m+1}_CRRP', f'{m+1}_CRRPP', 
                      f'{m+1}_GLDE', f'{m+1}_PLDE', f'{m+1}_DETA2_', f'{m+1}_CFQE2_', f'{m+1}_DFQES',
                      'id']
                df.columns=col
                module_df = pd.concat([module_df, df], axis=1, sort=False)
                #print("Path: {0:s}".format(f))
                print("j = {0:}, k = {1:}, input shape = {2:}, cavity shape = {3:}, time step = {4:3.2f} ms".format(j, k, df.shape, module_df.shape, tStep))
            module_df = module_df.loc[:,~module_df.columns.duplicated()]
            #y_tmp = pd.Series(log.cavity[j], index=[k])
            #y = y.append(y_tmp)
            
        else:
            print("Directory did not contain data files for all 8 cavities in the zone.")

        cavity_df = cavity_df.append(module_df)

j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 19), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 38), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 57), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 76), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 95), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 114), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 133), time step = 0.05 ms
j = 0, k = 1, input shape = (8192, 19), cavity shape = (8192, 152), time step = 0.05 ms


To reduce computational burden, we select only the (GMES, GASK, CRFP, DETA2) signals from each cavity (i.e. `<cavID>_<signal>`). (Note, the `CRFPP` signal was inadvertantly used in the training of the model, so we leave it in).

In [1614]:
sel_col = ["1_GMES", "1_GASK", "1_CRFP", "1_CRFPP", "1_DETA2_", "2_GMES", "2_GASK", "2_CRFP", "2_CRFPP", "2_DETA2_",
           "3_GMES", "3_GASK", "3_CRFP", "3_CRFPP", "3_DETA2_", "4_GMES", "4_GASK", "4_CRFP", "4_CRFPP", "4_DETA2_",
           "5_GMES", "5_GASK", "5_CRFP", "5_CRFPP", "5_DETA2_", "6_GMES", "6_GASK", "6_CRFP", "6_CRFPP", "6_DETA2_",
           "7_GMES", "7_GASK", "7_CRFP", "7_CRFPP", "7_DETA2_", "8_GMES", "8_GASK", "8_CRFP", "8_CRFPP", "8_DETA2_"]

In [1615]:
cavity_df = cavity_df[sel_col]

In [1616]:
cavity_df.head()

,1_GMES,1_GASK,1_CRFP,1_CRFPP,1_DETA2_,2_GMES,2_GASK,2_CRFP,2_CRFPP,2_DETA2_,...,7_GMES,7_GASK,7_CRFP,7_CRFPP,7_DETA2_,8_GMES,8_GASK,8_CRFP,8_CRFPP,8_DETA2_
0,18.0053,2.90009,2.99309,-116.884,-1.65894,18.5081,4.60938,3.84167,-104.980,-5.20752,...,17.0102,3.64624,1.50030,143.212,-2.73560,12.5082,2.99469,1.97090,-101.3320,-6.91040
1,18.0081,2.76337,2.92754,-116.856,-1.68640,18.5081,4.67651,3.77528,-105.606,-5.28992,...,17.0027,3.83789,1.76222,148.722,-2.40051,12.5028,3.15735,1.99497,-99.0088,-6.66321
2,18.0059,3.01025,2.86854,-114.274,-1.07666,18.5028,4.82574,3.71025,-103.063,-4.65820,...,16.9990,4.04114,1.51870,145.107,-3.12561,12.5082,3.10699,2.03230,-103.2550,-6.49292
3,18.0081,2.74963,2.95594,-117.202,-1.57104,18.5024,4.70093,3.83083,-103.629,-5.16907,...,16.9868,4.42383,1.71018,145.223,-3.59802,12.5038,3.13293,1.98448,-103.1840,-6.88843
4,18.0064,2.80609,2.96942,-113.967,-1.74133,18.5081,4.71588,3.81863,-105.980,-5.10315,...,16.9971,4.08417,1.56034,144.899,-3.58154,12.5097,3.07190,2.08105,-101.4150,-6.73462


In [1617]:
#cavity_df = cavity_df.rename(columns={"1_DETA2_":"1_DETA2", "2_DETA2_":"2_DETA2", "3_DETA2_":"3_DETA2", "4_DETA2_":"4_DETA2",
#                          "5_DETA2_":"5_DETA2", "6_DETA2_":"6_DETA2", "7_DETA2_":"7_DETA2", "8_DETA2_":"8_DETA2"});

A few diagnostic checks on the data.

In [1618]:
print("The input file has", cavity_df.shape[0], "rows and", cavity_df.shape[1], "columns")
print("The input file has", cavity_df.shape[0]/8192, "samples")

The input file has 8192 rows and 40 columns
The input file has 1.0 samples


### Feature Extraction

In [1619]:
from statsmodels.tsa.ar_model import AR
from sklearn import preprocessing

In [1620]:
def getStatARCoreffs(signals, maxLag):
    for i in range(0, np.shape(signals)[1]):
        model = AR(signals[:, i])
        model_fit = model.fit(maxlag=5, ic=None)
        if np.shape(model_fit.params)[0] < maxLag + 1:
            parameters = np.pad(model_fit.params, (0, maxLag + 1 - np.shape(model_fit.params)[0]), 'constant', constant_values=0)
        elif np.shape(model_fit.params)[0] > maxLag + 1:
            parameters = model_fit.params[: maxLag]
        else:
            parameters = model_fit.params

        if i == 0:
            coefficients = parameters
        else:
            coefficients = np.append(coefficients, parameters, axis=0)

    return pd.DataFrame(coefficients).T

In [1621]:
signalScaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True)
cavity_df[sel_col] = signalScaler.fit_transform(cavity_df[sel_col])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [1622]:
X_cavity_master = getStatARCoreffs(cavity_df.values, maxLag=5)

In [1623]:
X_cavity_master.shape

(1, 240)

A few diagnostic checks.

In [1624]:
print("Number of training examples: {}".format(X_cavity_master.shape[0]))
print("Number of features: {}".format(X_cavity_master.shape[1]))

Number of training examples: 1
Number of features: 240


Use `joblib` to import previously saved model for determining which cavity tripped. The model is based on a `RandomForestClassifier` using the following parameters found using `GridSearchCV`:
- `n_estimators` = X
- `max_depth` = X
- `min_samples_split` = X
- `max_features` = None

In [1625]:
from sklearn.externals import joblib

RF_cavity = joblib.load('RF_CAVITY_AR_02202020.sav')

In [1626]:
cavityID = RF_cavity.predict(X_cavity_master)
cavityID_prob = RF_cavity.predict_proba(X_cavity_master)
cavityID_str = cavityID.astype(str)[0]
print("Cavity", cavityID_str, "was the first to go unstable")
ID_confidence = float(cavityID_prob[0][cavityID]*100)
print("Confidence level is:", ID_confidence,"%")

Cavity 2 was the first to go unstable
Confidence level is: 85.21794871794872 %


## Which Fault?

Once again, to make the feature engineering manageable (i.e. less computationally expensive) we do the following:

- for each of the 17 signals from the first faulted cavity (prediction from previous model) use `tsfresh` to compute a subset of available features
    - specifically, compute the top contributing features using the `feature_importances_`

### Reading Data

Pass `cavityID` - from above - into the script below to load all 17 signals from _only the cavity that tripped_.

In [1627]:
fault_df = pd.DataFrame(columns=['Time', 'IMES', 'QMES', 'GMES', 'PMES', 'IASK', 'QASK', 'GASK',
                                  'PASK', 'CRFP', 'CRFPP', 'CRRP', 'CRRPP', 'GLDE', 'PLDE', 'DETA2', 'CFQE2', 'DFQES', 'id'])

In some instances, the time stamp changes enough from one cavity to the next that you may need to use `time_format[:3]`

In [1628]:
k = 0

for i in filelist:
    data_file_path =dir/i

    log = pd.read_table(data_file_path, sep='\t')

    m, n = log.shape

    for j in range(0, m):
        k += 1
        date, time = log.time[j].split(" ", 1)

        date_format = date.replace("/", "_")
        time_format = time.replace(":", "")

        list1 = [time_format, '.', '?']

        ct = os.path.join(module_path, log.zone[j], date_format, "".join(list1))

        list2 = (cav_dict[log.zone[j]], cavityID_str, 'WFSharv.',
                 date_format, '_', time_format[:3], '*', '.?.txt')

        filename = "".join(list2)
        #print(filename)

        for f in glob.glob(os.path.join(ct, filename)):
            df = pd.read_table(f, sep='\t')
            sLength = len(df['Time'])
            tStep = (df.Time[2] - df.Time[1]) # in milliseconds
            df['id'] = pd.Series(k, index=df.index)
            df.columns = fault_df.columns
            fault_df = fault_df.append(df)
            print("Path: {0:s}".format(f))
            print("j = {0:}, k = {1:}, input shape = {2:}, fault shape = {3:}, time step = {4:3.2f} ms".format(j, k, df.shape, fault_df.shape, tStep))

Path: D:\RF WAVEFORMS\rfw_tsf_extractor-Spring-2019\waveform-data\rf\2L26\2019_04_14\100331.9\R2Q2WFSharv.2019_04_14_100331.9.txt
j = 0, k = 1, input shape = (8192, 19), fault shape = (8192, 19), time step = 0.05 ms


In [1629]:
print("The input file has", fault_df.shape[0], "rows and", fault_df.shape[1], "columns")
print("The input file has", fault_df.shape[0]/8192, "samples")

The input file has 8192 rows and 19 columns
The input file has 1.0 samples


To reduce the computational load only the *top 50* features - based on previous analysis of a `RandomForestClassifier` - are computed and used as inputs.

In [1630]:
top_features = ['GMES__augmented_dickey_fuller__attr_"usedlag"',
 'CRFP__maximum',
 'CRFP__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"stderr"',
 'GASK__fft_coefficient__coeff_64__attr_"abs"',
 'CRFP__augmented_dickey_fuller__attr_"teststat"',
 'GMES__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',
 'CRFP__augmented_dickey_fuller__attr_"pvalue"',
 'QMES__augmented_dickey_fuller__attr_"usedlag"',
 'GMES__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',
 'PLDE__augmented_dickey_fuller__attr_"usedlag"',
 'CRFP__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
 'CRFP__fft_coefficient__coeff_96__attr_"abs"',
 'GMES__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
 'IMES__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',
 'PLDE__partial_autocorrelation__lag_5',
 'PLDE__minimum',
 'GMES__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',
 'QMES__approximate_entropy__m_2__r_0.5',
 'PLDE__max_langevin_fixed_point__m_3__r_30',
 'QASK__fft_coefficient__coeff_17__attr_"abs"',
 'CRFP__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"slope"',
 'CRFP__fft_coefficient__coeff_48__attr_"abs"',
 'GMES__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',
 'GMES__number_cwt_peaks__n_1',
 'CRFP__ar_coefficient__k_10__coeff_0',
 'QMES__approximate_entropy__m_2__r_0.7',
 'GASK__fft_aggregated__aggtype_"skew"',
 'CRRP__longest_strike_above_mean',
 'PLDE__partial_autocorrelation__lag_6',
 'IMES__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',
 'PLDE__fft_coefficient__coeff_82__attr_"real"',
 'CRFP__ratio_beyond_r_sigma__r_1.5',
 'PLDE__friedrich_coefficients__m_3__r_30__coeff_0',
 'IMES__fft_coefficient__coeff_11__attr_"abs"',
 'GASK__ratio_beyond_r_sigma__r_6',
 'GASK__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
 'QASK__fft_aggregated__aggtype_"skew"',
 'IMES__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',
 'IASK__fft_aggregated__aggtype_"skew"',
 'DETA2__longest_strike_above_mean',
 'CRRPP__quantile__q_0.9',
 'GASK__fft_aggregated__aggtype_"centroid"',
 'QASK__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
 'PLDE__partial_autocorrelation__lag_2',
 'DFQES__ratio_value_number_to_time_series_length',
 'PLDE__fft_coefficient__coeff_43__attr_"angle"',
 'CRFP__percentage_of_reoccurring_datapoints_to_all_datapoints',
 'CRFP__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',
 'QASK__spkt_welch_density__coeff_5',
 'PLDE__autocorrelation__lag_9']

In [1631]:
import tsfresh
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters, MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings

In [1632]:
extraction_settings = settings.from_columns(top_features)

# per https://github.com/blue-yonder/tsfresh/issues/478
%time X_fault = extract_features(fault_df.astype("float64"), column_id="id", column_sort="Time", \
                           impute_function=impute, kind_to_fc_parameters=extraction_settings, default_fc_parameters={})

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 17/17 [00:11<00:00,  1.47s/it]
 'PLDE__max_langevin_fixed_point__m_3__r_30'] did not have any finite values. Filling with zeros.


Wall time: 12.1 s


In [1633]:
X_fault_master = X_fault[top_features]
X_fault_master.shape

(1, 50)

This step is required because the `RF_FAULT_top50` model was trained on standardized data.

In [1634]:
X_fault_mean = np.load('RF_FAULT_top50_mean.npy')
X_fault_var  = np.load('RF_FAULT_top50_var.npy')

In [1635]:
X_fault_master = (X_fault_master - X_fault_mean) / X_fault_var

Load the `RF_FAULT_top50` model.

In [1636]:
RF_fault = joblib.load('RF_FAULT_top50_01292020.sav')

Load the `preprocessing` library to apply the `inverse_transform` to the numerical result and return a categorical label.

In [1637]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.classes_ = np.load('le_fault_classes.npy')
le.classes_

array(['Controls Fault', 'E_Quench', 'Heat Riser Choke', 'Microphonics',
       'Multi Cav turn off', 'Quench_100ms', 'Quench_3ms',
       'Single Cav Turn off'], dtype=object)

In [1638]:
cavityFault = RF_fault.predict(X_fault_master)
cavityFault_prob = RF_fault.predict_proba(X_fault_master)
cavityFault_name = le.inverse_transform(cavityFault)
cavityFault_name_str = cavityFault_name.astype(str)[0]

print("The trip was caused by a", cavityFault_name_str, "fault")
fault_confidence = float(cavityFault_prob[0][cavityFault]*100)
print("Confidence level is:", fault_confidence,"%")

The trip was caused by a Single Cav Turn off fault
Confidence level is: 100.0 %


## Summary

In [1639]:
print("The trip event was initated by a", cavityFault_name_str ,"fault in cavity", int(cavityID), "of zone", log.zone[0])

The trip event was initated by a Single Cav Turn off fault in cavity 2 of zone 2L26


In [1640]:
print('cavity', cavityID_str, '(',round(ID_confidence,2), ')', cavityFault_name_str, '(',round(fault_confidence,2),')')

cavity 2 ( 85.22 ) Single Cav Turn off ( 100.0 )


In [1641]:
print("Executing the notebook took:", datetime.now() - startTime, "(h:mm:ss)")

Executing the notebook took: 0:00:13.109043 (h:mm:ss)


In [1646]:
#import tsfresh
#print(tsfresh.__version__)
#from platform import python_version
#print(python_version())
#print(pd.__version__)
#print(np.__version__)
#import sklearn
#sklearn.__version__

In [1648]:
#!pip list